# Training the Koopman embedding models for the Gissinger system

In [1]:
%load_ext autoreload
%autoreload 2

from trphysx.config.configuration_auto import AutoPhysConfig
from trphysx.embedding.embedding_auto import AutoEmbeddingModel
from trphysx.embedding.training import *
from trphysx.embedding import embedding_gissinger as EG


import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import matplotlib.pyplot as plt
import numpy as np
import os
import h5py
import argparse
import logging

%matplotlib inline

### Passing arguments

In [2]:
data_dir = "../data/gissinger/"


argv = []
argv = argv + ["--exp_name", "gissinger"]
argv = argv + [
    "--training_h5_file",
     data_dir + "gissinger_training_rk.hdf5"]
argv = argv + [
    "--eval_h5_file", 
    data_dir + "gissinger_valid_rk.hdf5"
                ]
argv = argv + [
    "--exp_dir",
    "./outputs/embedding_gissinger"
     ]
argv = argv + ["--batch_size", '512']
argv = argv + ["--block_size", "16"]
argv = argv + ["--n_train", "2048"]
argv = argv + ["--n_eval", "64"]
argv = argv + ["--epochs", "200"]


args = EmbeddingParser().parse(args = argv)


### Creating Physical configuration and getting handler

In [3]:
config = AutoPhysConfig.load_config(args.exp_name)
dataloader = AutoDataHandler.load_data_handler(args.exp_name)


## Creating training and testing dataloader

In [4]:
training_loader = dataloader.createTrainingLoader(
    args.training_h5_file, 
    block_size=args.block_size, 
    stride=args.stride, 
    ndata=args.n_train, 
    batch_size=args.batch_size)
testing_loader = dataloader.createTestingLoader(
    args.eval_h5_file, 
    block_size=32, 
    ndata=args.n_eval, 
    batch_size=8)



/home/mirksonius/Desktop/seminar_fizika/kod/trphysx/embedding/training/enn_data_handler.py:239: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:230.)
  data_series = torch.Tensor(f[key])


### Getting devices for training

In [5]:
if(torch.cuda.is_available()):
    use_cuda = "cuda"
args.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("Torch device:{}".format(args.device))

Torch device:cuda:0


### Initializing and loading model

In [6]:
model = AutoEmbeddingModel.init_trainer(args.exp_name, config).to(args.device)
mu, std = dataloader.norm_params
model.embedding_model.mu = mu.to(args.device)
model.embedding_model.std = std.to(args.device)
if args.epoch_start > 1:
  model.load_model(args.ckpt_dir, args.epoch_start)

### Setting up Logger

In [7]:
logging.basicConfig(
    format="%(asctime)s - %(levelname)s - %(name)s -   %(message)s",
    datefmt="%m/%d/%Y %H:%M:%S",
    level=logging.INFO)

## Defining opttimizers and learning scheduler

In [8]:
optimizer = torch.optim.Adam(model.parameters(), lr=args.lr*0.995**(args.epoch_start-1), weight_decay=1e-8)
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.995)

## Training and validating model
Validation is perfomed every 5 epochs on a separate dataset.

In [9]:
trainer = EmbeddingTrainer(model, args, (optimizer, scheduler))
trainer.train(training_loader, testing_loader)

124it [00:11, 10.98it/s]
01/16/2023 18:20:57 - INFO - trphysx.embedding.training.enn_trainer -   Epoch 1: Training loss 571466.062, Lr 0.00100
01/16/2023 18:20:57 - INFO - trphysx.embedding.training.enn_trainer -   Epoch 1: Test loss: 0.00
124it [00:08, 14.50it/s]
01/16/2023 18:21:05 - INFO - trphysx.embedding.training.enn_trainer -   Epoch 2: Training loss 20755.508, Lr 0.00099
124it [00:09, 12.69it/s]
01/16/2023 18:21:15 - INFO - trphysx.embedding.training.enn_trainer -   Epoch 3: Training loss 18191.412, Lr 0.00099
124it [00:09, 13.02it/s]
01/16/2023 18:21:25 - INFO - trphysx.embedding.training.enn_trainer -   Epoch 4: Training loss 12170.416, Lr 0.00099
124it [00:09, 13.54it/s]
01/16/2023 18:21:34 - INFO - trphysx.embedding.training.enn_trainer -   Epoch 5: Training loss 10104.886, Lr 0.00098
01/16/2023 18:21:34 - INFO - trphysx.embedding.training.enn_trainer -   Epoch 5: Test loss: 0.00
124it [00:09, 13.00it/s]
01/16/2023 18:21:43 - INFO - trphysx.embedding.training.enn_trainer - 